### Imports

In [ ]:
import pandas as pd
import datetime as dt

matches = pd.read_csv("matches.csv", index_col=0)

### Data Cleaning and Feature Engineering

In [ ]:
matches.head()

In [ ]:
matches.columns

In [ ]:
matches.drop(["round", 
              "comp", 
              "season", 
              "attendance", 
              "notes", 
              "captain", 
              "formation", 
              "referee", 
              "match report", 
              "notes"], 
             axis=1, inplace=True)

In [ ]:
matches["team"].value_counts()

In [ ]:
#convert categoric variables to numeric for machine learning model

In [ ]:
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [ ]:
matches["day_code"] = matches["date"].dt.dayofweek

In [ ]:
#retrieve just hour from ko time - time of day may affect performance
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype(int)

In [ ]:
#create dict to rename teams so they match in home/away column
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {  
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Sheffield United": "Sheffield Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Bromwich Albion": "West Brom",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves",    
}
mapping = MissingDict(**map_values)

In [ ]:
matches['team'] = matches['team'].map(mapping)

In [ ]:
matches.head()

In [ ]:
#determine points acquired
matches["points"] = matches["result"].apply(lambda row: 3 if row=="W" else 1 if row=="D" else 0)

In [ ]:
#convert W/L/D to numbers for classification
matches["results_class"] = matches["result"].apply(lambda row: 2 if row=="W" else 1 if row=="D" else 0)

In [ ]:
matches.columns

In [ ]:
matches.sort_values('date', inplace=True)

In [ ]:
#create rolling averages based on previous 4 games
cols = ['points', 'gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'poss', 'xg', 'xga']
new_cols = [f"{c}_rolling" for c in cols]
matches[new_cols] = matches.groupby('team')[cols].transform(lambda x: x.rolling(4).mean().shift().bfill())

In [ ]:
#group by team
grp_matches = matches.groupby("team").apply(lambda a: a[:]).drop('team', axis=1).droplevel(1)

In [ ]:
grp_matches.reset_index(inplace=True)

In [ ]:
#keep important columns that have potential impact on performance
avg_matches = grp_matches[['date', 
                           'team',
                           'opponent',
                           'venue_code', 
                           'hour', 
                           'day_code', 
                           'points_rolling', 
                           'gf_rolling', 
                           'ga_rolling', 
                           'sh_rolling', 
                           'sot_rolling', 
                           'dist_rolling', 
                           'fk_rolling', 
                           'pk_rolling', 
                           'pkatt_rolling',
                           'poss_rolling',
                           'xg_rolling', 
                           'xga_rolling', 
                           'results_class']].dropna(axis=0)

In [ ]:
#split in to home and awya matches based on venue code
home_matches = avg_matches[avg_matches["venue_code"]==1].sort_values("date")
away_matches = avg_matches[avg_matches["venue_code"]==0].sort_values("date")

In [ ]:
home_matches.head()

In [ ]:
away_matches.head()

In [ ]:
#remerge so no matches are repeated
merge_matches = pd.merge(home_matches, away_matches, 
                         left_on=["date", "team", "opponent"], 
                         right_on=["date", "opponent", "team"], 
                         suffixes=('_home', '_away')).sort_values("date")

In [ ]:
merge_matches.drop(["opponent_home", "opponent_away", "venue_code_home", "venue_code_away", "results_class_away", "hour_away", "day_code_away"], axis=1, inplace=True)

In [ ]:
merge_matches.columns

In [ ]:
merge_matches.rename({"hour_home": "hour", "day_code_home": "day_code"}, axis=1, inplace=True)

In [ ]:
merge_matches.columns

In [ ]:
merge_matches["team_home_code"] = merge_matches["team_home"].astype("category").cat.codes

In [ ]:
merge_matches["team_away_code"] = merge_matches["team_away"].astype("category").cat.codes

In [ ]:
# create columns with average stat differences between the two teams
merge_matches['points_rolling_diff'] = (merge_matches['points_rolling_home']-merge_matches['points_rolling_away'])
merge_matches['gf_rolling_diff'] = (merge_matches['gf_rolling_home']-merge_matches['gf_rolling_away'])
merge_matches['ga_rolling_diff'] = (merge_matches['ga_rolling_home']-merge_matches['ga_rolling_away'])
merge_matches['sh_rolling_diff'] = (merge_matches['sh_rolling_home']-merge_matches['sh_rolling_away'])
merge_matches['sot_rolling_diff'] = (merge_matches['sot_rolling_home']-merge_matches['sot_rolling_away'])
merge_matches['poss_rolling_diff'] = (merge_matches['poss_rolling_home']-merge_matches['poss_rolling_away'])
merge_matches['xg_rolling_diff'] = (merge_matches['xg_rolling_home']-merge_matches['xg_rolling_away'])
merge_matches['xga_rolling_diff'] = (merge_matches['xga_rolling_home']-merge_matches['xga_rolling_away'])

In [ ]:
#final column features
final_df = merge_matches[['date',
                          'hour', 
                          'day_code', 
                          'team_home',
                          'team_away', 
                          'points_rolling_diff', 
                          'gf_rolling_diff', 
                          'ga_rolling_diff', 
                          'sh_rolling_diff', 
                          'sot_rolling_diff', 
                          'poss_rolling_diff', 
                          'xg_rolling_diff', 
                          'xga_rolling_diff', 
                          'results_class_home']]

In [ ]:
#observe correlations between features and target 
import seaborn as sns

sns.heatmap(final_df.corr()[['results_class_home']].sort_values("results_class_home", ascending=False), annot=True, cmap="mako", vmax=1, vmin=-1)

In [ ]:
final_df.head()

In [ ]:
#convert team names to numeric codes for machine learning model
final_df["team_home_code"] = final_df["team_home"].astype("category").cat.codes
final_df["team_away_code"] = final_df["team_away"].astype("category").cat.codes

## Machine Learning Model

In [ ]:
#train = final_df[final_df["date"] < "2022-01-01"].select_dtypes(['number'])

In [ ]:
#test = final_df[final_df["date"] > "2022-01-01"].select_dtypes(['number'])

In [ ]:
#select only numeric features for model
final_df = final_df.select_dtypes(['number'])

In [ ]:
final_df.head()

In [ ]:
# test/train split based on time - 67% train 33% test
import numpy as np 

train, test = np.split(final_df, [int(.67*len(final_df))])

In [ ]:
#normalise feature values
# from sklearn import preprocessing

# d = preprocessing.normalize(final_df.drop("results_class_home", axis=1), axis=0)
# scaled_df = pd.DataFrame(d, columns=final_df.drop("results_class_home", axis=1).columns)
# scaled_df = scaled_df.join(final_df["results_class_home"])

In [ ]:
# scaled_df.head()

In [ ]:
train

In [ ]:
test

In [ ]:
# from sklearn.model_selection import train_test_split

#test/train split - stratified sampling (equal weights of each class in test/train)
# X_train, X_test, y_train, y_test = train_test_split(final_df.drop("results_class_home", axis=1), final_df["results_class_home"], test_size=0.2, stratify = final_df["results_class_home"])

In [ ]:
X_train = train.drop("results_class_home", axis=1)
y_train = train["results_class_home"]

In [ ]:
X_test = test.drop("results_class_home", axis=1)
y_test = test["results_class_home"]

In [ ]:
X_train.head()

In [ ]:
X_train.columns[1]

In [ ]:
#viusalise general relationship between variables and target
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(18, 10))
ax.set_xlabel('Target', fontsize=10)
ax.set_ylabel('Feature', fontsize='medium')

x=y_train

for i in range(len(X_train.columns)):
    y=X_train.iloc[:,i]
    plt.scatter(y=y, x=x, label=X_train.columns[i])
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x, p(x))
    
ax.legend(loc='best', ncol=8)

In [ ]:
import xgboost as xgb
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll import scope
import numpy as np

In [ ]:
def getBestModelfromTrials(trials):
    valid_trial_list = [trial for trial in trials
                            if STATUS_OK == trial['result']['status']]
    losses = [ float(trial['result']['loss']) for trial in valid_trial_list]
    index_having_minumum_loss = np.argmin(losses)
    best_trial_obj = valid_trial_list[index_having_minumum_loss]
    return best_trial_obj['result']['model']

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score

### XGBoost Regression

In [ ]:
# xgbreg = xgb.XGBRegressor(objective="reg:squarederror", 
#                           random_state=42
#                          )

In [ ]:
# xgbreg.fit(X_train, y_train)

In [ ]:
# y_pred = xgbreg.predict(X_test)

In [ ]:
# len(y_test)

In [ ]:
# len(y_pred)

In [ ]:
# len(X_train.columns)

In [ ]:
# from sklearn.metrics import r2_score

In [ ]:
# r2_score(y_test, y_pred)

In [ ]:
# def objective_xgbreg(space):
    
#     model = xgb.XGBRegressor(objective="reg:squarederror", 
#                              max_depth=int(space['max_depth']),
#                              min_child_weight=int(space['min_child_weight']),
#                              n_estimators=int(space['n_estimators']),
#                              eval_metric="rmse",
#                              early_stopping_rounds=10,
#                              random_state=42
#                             )
    
#     evaluation=[(X_test, y_test)]
    
#     model.fit(X_train, y_train, eval_set=evaluation, verbose=False)
    
#     y_pred = model.predict(X_test)
#     score = r2_score(y_test, y_pred)
    
#     loss = 1 - score
    
#     return {'loss': loss, 'status': STATUS_OK, 'model': model}

In [ ]:
# space={'max_depth': hp.quniform("max_depth", 1, 18, 1),
#        'min_child_weight' : hp.quniform('min_child_weight', 0, 1000, 1),
#        'n_estimators': hp.quniform("n_estimators", 1, 1000, 1)
#       }

In [ ]:
# xgbreg_trials = Trials()

In [ ]:
# best_params_xgb = fmin(
#     fn=objective_xgbreg,
#     space=space,
#     algo=tpe.suggest,
#     trials=xgb_trials,
#     max_evals=1000)

In [ ]:
# print(best_params_xgb)

In [ ]:
# xgbreg_model = getBestModelfromTrials(xgbreg_trials)

In [ ]:
# pred = xgbreg_model.predict(X_test)

In [ ]:
# pred_rnd = np.around(pred)

In [ ]:
# r2_score(y_test, pred)

In [ ]:
# xgbreg_model

### XGBoost Classification

In [ ]:
xgbclf = xgb.XGBClassifier(objective="multi:softmax")

In [ ]:
xgbclf.fit(X_train, y_train)

In [ ]:
clf_pred = xgbclf.predict(X_test)

In [ ]:
accuracy_score(y_test, clf_pred)

In [ ]:
def objective_xgbclf(space):
    
    model = xgb.XGBClassifier(objective="multi:softmax", 
                              num_class=3,
                              max_depth=space['max_depth'],
                              min_child_weight=space['min_child_weight'],
                              n_estimators=space['n_estimators'],
                              gamma=space['gamma'],
                              learning_rate=space['learning_rate'],
                              reg_lambda=space['reg_lambda'],
                              eval_metric="mlogloss",
                              early_stopping_rounds=space['early_stopping_rounds'],
                              subsample=space['subsample']
                             )
    
    evaluation=[(X_test, y_test)]
    
    model.fit(X_train, y_train, eval_set=evaluation, verbose=False)
    
    y_pred_probs = model.predict_proba(X_test)
    y_pred = model.predict(X_test)
    score = cohen_kappa_score(y_test, y_pred)
    
    loss = 1-score
    
    return {'loss': loss, 'status': STATUS_OK, 'model': model}

In [ ]:
space={'max_depth': scope.int(hp.quniform("max_depth", 1, 10, 1)),
       'min_child_weight': scope.int(hp.quniform('min_child_weight', 1, 100, 5)),
       'n_estimators': scope.int(hp.quniform("n_estimators", 1, 1000, 50)),
       'gamma': hp.quniform('gamma', 0, 1, 0.05),
       'learning_rate': hp.quniform('learning_rate', 0.01, 0.2, 0.005),
       'reg_lambda': hp.choice('reg_lambda', [0.1, 1.0, 5.0, 10.0, 50.0, 100.0]),
       'early_stopping_rounds': hp.quniform('early_stopping_rounds', 10, 100, 5),
       'subsample': hp.quniform('subsample', 0.5, 1, 0.05)
      }

In [ ]:
xgbclf_trials = Trials()

In [ ]:
best_params_xgbclf = fmin(
    fn=objective_xgbclf,
    space=space,
    algo=tpe.suggest,
    trials=xgbclf_trials,
    max_evals=2000)

In [ ]:
best_params_xgbclf

In [ ]:
"""
{'early_stopping_rounds': 45.0,
 'gamma': 0.1,
 'learning_rate': 0.11,
 'max_depth': 5.0,
 'min_child_weight': 0.0,
 'n_estimators': 150.0,
 'reg_lambda': 1,
 'subsample': 0.65}
 accuracy: 0.5357142857142857
"""

In [ ]:
xgbclf = getBestModelfromTrials(xgbclf_trials)

In [ ]:
evaluation=[(X_train, y_train), (X_test, y_test)]
xgbclf.fit(X_train, y_train, eval_set=evaluation, verbose=False)

In [ ]:
pred = xgbclf.predict(X_test)

In [ ]:
pred

In [ ]:
accuracy_score(y_test, pred)

In [ ]:
xgbclf.best_ntree_limit

In [ ]:
results = xgbclf.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["mlogloss"], label="Training loss")
plt.plot(results["validation_1"]["mlogloss"], label="Validation loss")
plt.axvline(xgbclf.best_ntree_limit, color="gray", label="Optimal tree number")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

In [ ]:
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt

In [ ]:
sort = xgbclf_model.feature_importances_.argsort()
plt.barh(X_train.columns[sort], xgbclf_model.feature_importances_[sort])
plt.xlabel("Feature Importance")

### Multi Classifier Model Test

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
names = ["Nearest Neighbors", "Logistic Regression","Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    LogisticRegression(),
    SVC(kernel="linear", C=0.025, probability=True),
    SVC(gamma=2, C=1, probability=True),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = cohen_kappa_score(y_test, y_pred)

    # prediction_proba = clf.predict_proba(X_test)
    # logloss = log_loss(y_test,prediction_proba)
    # precision, recall, fscore, support = score(y_test, prediction)
    # conf_martrix = confusion_matrix(y_test, prediction)
    # clas_report = classification_report(y_test, prediction)

    print(name, score)

### Linear SVM Optimisation

In [ ]:
# lin_svm = SVC(kernel="linear", C=0.025, probability=True)

In [ ]:
# lin_svm.fit(X_train, y_train)

In [ ]:
# svm_pred = lin_svm.predict(X_test)

In [ ]:
# accuracy_score(y_test, svm_pred)

In [ ]:
# def objective_svm(space):
    
#     model = SVC(C=space['C'],
#                 gamma=space['gamma'],
#                 degree=space['degree'],
#                 kernel='linear',
#                 probability=True
#                )
    
#     model.fit(X_train, y_train)
    
#     pred = model.predict(X_test)
#     score = cohen_kappa_score(y_test, pred)
    
#     loss = 1 - score
    
#     return {'loss': loss, 'status': STATUS_OK, 'model': model}

In [ ]:
# space={'C': hp.loguniform('C', 0, 1),
#        'gamma' : hp.loguniform('gamma', 0, 1),
#        'degree': scope.int(hp.choice('degree', [1, 2, 3, 5]))
#       }

In [ ]:
# svm_trials = Trials()

In [ ]:
# best_params_svm = fmin(
#     fn=objective_svm,
#     space=space,
#     algo=tpe.suggest,
#     trials=svm_trials,
#     max_evals=200)

In [ ]:
# svm_model = getBestModelfromTrials(svm_trials)

In [ ]:
# pred = svm_model.predict(X_test)

In [ ]:
# pred

In [ ]:
# accuracy_score(y_test, pred)

In [ ]:
# svm_model